In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

## Data completeness

In [31]:
df_clicks_train = pd.read_parquet("c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/reindexed_data/influence_data/df_clicks_train_unique_idea_indexed.parquet")
df_carts_train = pd.read_parquet('c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/reindexed_data/influence_data/df_carts_train_unique_idea.parquet')
#df_orders_train = pd.read_parquet('c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/reindexed_data/idea_data/df_orders_train_unique.parquet')

In [3]:
#df_clicks_train.to_parquet('reindexed_data/df_clicks_train_vs_encoded')
#df_carts_train.to_parquet('reindexed_data/df_carts_train')
#df_orders_train.to_parquet('reindexed_data/df_orders_train')

In [4]:
import sys
extra_path = 'c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita'
sys.path.append(extra_path)

In [32]:
df_clicks_t = pd.read_parquet("c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/clicks/test")
df_carts_t = pd.read_parquet("c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/carts/test")
df_orders_t = pd.read_parquet("c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/orders/test")

In [4]:
df_carts_train.head()

,session,aid,ts,session_encoded,aid_encoded,res_encoded,res_encoded_tfidf,pred,pred_tfidf
0,1,105393,1.659991e+12,0,11377,"[11377, 23133, 115794, 92048, 69572, 85212, 11...","[11377, 23133, 115794, 92048, 112407, 85212, 6...","[105393.0, 215311.0, 1073464.0, 854637.0, 6472...","[105393.0, 215311.0, 1073464.0, 854637.0, 1042..."
1,3,812246,1.659997e+12,1,87413,"[106143, 3129, 76896, 80571, 179814, 13799, 93...","[106143, 76896, 3129, 13799, 136268, 45799, 59...","[984459.0, 29160.0, 714724.0, 748586.0, 166220...","[984459.0, 714724.0, 29160.0, 128326.0, 126082..."
2,11,1528062,1.660210e+12,2,165310,"[123638, 12161, 36924, 148847, 129961, 92826, ...","[123638, 129961, 12161, 148847, 92826, 21690, ...","[1145803.0, 113002.0, 345724.0, 1375870.0, 120...","[1145803.0, 1203086.0, 113002.0, 1375870.0, 86..."
3,13,139863,1.659736e+12,3,15080,[],[],[],[]
4,14,200631,1.660049e+12,4,21510,"[21510, 100049, 146423, 57619, 145262, 21798, ...","[21510, 57619, 39184, 146423, 174252, 100049, ...","[200631.0, 928901.0, 1353740.0, 536378.0, 1343...","[200631.0, 536378.0, 366603.0, 1353740.0, 1611..."


## Return predictions

In [5]:
def set_ranks(df: pd.DataFrame):
    df_c = df.copy()
    df_c["rank"] = df_c.groupby("session")["ts"].rank(method="first", ascending=True)
    return df_c[["session", "aid", 'rank']]

In [6]:
#df_clicks_r = set_ranks(df_clicks_train)
df_carts_r = set_ranks(df_carts_train)
#df_orders_r = set_ranks(df_orders_train)

In [7]:
def rank_stat(df_r: pd.DataFrame, ks: list):
    df_mr = df_r.groupby("session").agg("max")["rank"]
    for k in ks:
        cnt_me = len(df_mr[df_mr >= k])
        print(f"Users with more than {k} test items: {cnt_me} ({round(cnt_me / len(df_mr) * 100, 2)}%)")

In [8]:
rank_stat(df_carts_r, [3, 5, 10, 20])

Users with more than 3 test items: 0 (0.0%)
Users with more than 5 test items: 0 (0.0%)
Users with more than 10 test items: 0 (0.0%)
Users with more than 20 test items: 0 (0.0%)


In [9]:
rank_stat(df_carts_r, [3, 5, 10, 20])

Users with more than 3 test items: 0 (0.0%)
Users with more than 5 test items: 0 (0.0%)
Users with more than 10 test items: 0 (0.0%)
Users with more than 20 test items: 0 (0.0%)


In [10]:
#rank_stat(df_orders_r, [3, 5, 10, 20])

# Metrics

In [11]:
K_RECS = 200
K_TEST = 5

In [34]:
df_carts_r = pd.read_parquet('c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/clicks/test')

In [13]:
df_carts_r

,session,aid,ts,type
597,5,1813405,1660347708319,carts
1003,13,284722,1660566004531,carts
1005,13,1662866,1660566043771,carts
1007,13,1177411,1660566074020,carts
1012,13,1670763,1660566927761,carts
...,...,...,...,...
1351936,327670,669555,1660514505932,carts
1352254,327677,1157882,1660341835034,carts
1352256,327677,855613,1660342258813,carts
1352257,327677,904999,1660343028183,carts


In [14]:
#df_carts_pred_final = df_carts_r[['session', 'aid', 'pred']]

In [15]:
def extract_n_elements(row, n):
    return np.pad(row[:n], (0, max(n - len(row), 0)), mode='constant', constant_values=0)



In [16]:
def prepare_data(df_test, df_recs, k_test, k_recs):
    valid_session = np.intersect1d(
        df_test["session"].unique(),
        df_recs["session"].unique()
    )

    df_test_l = set_ranks(df_test)
    df_test_l = df_test_l[df_test_l["session"].isin(valid_session)]
    df_test_l = df_test_l[df_test_l["rank"] <= k_test]
    df_test_l = df_test_l.groupby("session")["aid"].apply(list).reset_index().sort_values("session")

    df_test_l["aid"] = (
        df_test_l["aid"]
        .apply(lambda x: np.pad(
            x[:k_test],
            (0, max(k_test - len(x), 0)),
            mode="constant",
            constant_values=-1
        ))
    )
    test = np.stack(df_test_l["aid"].values, axis=0)

    df_recs_l = df_recs[df_recs["session"].isin(valid_session)]
    df_recs_l["modified_pred"] = (
        df_recs_l["pred"]
        .apply(lambda x: extract_n_elements(x, k_recs))
    )
    
    recs = df_recs_l["modified_pred"].tolist()

    return test, recs


In [17]:
test_carts, recs_carts = prepare_data(df_carts_r, df_carts_train, K_TEST, K_RECS)

C:\Users\tanni\AppData\Local\Temp\ipykernel_9960\3645500487.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recs_l["modified_pred"] = (


In [35]:
test_carts, recs_carts = prepare_data(df_carts_r, df_clicks_train, K_TEST, K_RECS)

C:\Users\tanni\AppData\Local\Temp\ipykernel_9960\3645500487.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recs_l["modified_pred"] = (


In [36]:
import numpy as np

class Metrics:
    def __init__(self, k, recommendations, holdout):
        self.k = k
        self.recommendations = recommendations[:, :self.k]
        self.holdout = holdout

    def hit_rate(self):
        return np.array(
            [
                np.intersect1d(self.holdout[i], self.recommendations[i]).any()
                for i in range(self.holdout.shape[0])
            ]
        ).mean()

    def mean_reciprocal_rank(self):
        n_test_users = self.holdout.shape[0]
        hits_rank = [
            np.where(np.in1d(self.holdout[i], self.recommendations[i]))[0]
            for i in range(self.holdout.shape[0])
        ]
        hits_rank = np.array([el[0] for el in hits_rank if el.shape[0]])
        return (1. / (hits_rank + 1)).sum() / n_test_users

    def recall(self):
        numerator = np.sum(
            [
                np.intersect1d(self.holdout[i], self.recommendations[i]).shape[0]
                for i in range(self.holdout.shape[0])
            ]
        )
        denominator = np.sum([min(self.k, self.holdout[i].shape[0]) for i in range(self.holdout.shape[0])])
        return numerator / denominator


In [37]:
import numpy as np


class Metrics:
    def __init__(
            self,
            k: int,
            recommendations: np.ndarray,
            holdout: np.ndarray
            ):
        self.k = k
        self.recommendations = recommendations[:, :k]
        self.holdout = holdout

    def report(self):
        print(f"HR@{self.k} = {round(self.hit_rate(), 4)}")
        print(f"MRR@{self.k} = {round(self.mean_reciprocal_rank(), 4)}")
        print(f"Recall@{self.k} = {round(self.recall(), 4)}")

    def hit_rate(self):
        return (
            self._get_hit_mask()
            .any(axis=1)
            .mean()
        )

    def mean_reciprocal_rank(self):
        hits_mask = self._get_hit_mask()

        idx = np.argwhere(hits_mask.argmax(axis=1)).squeeze(axis=1)

        return np.sum(
            1 / (hits_mask[idx].argmax(axis=1) + 1)
        ) / hits_mask.shape[0]

    def recall(self):
        hits_mask = self._get_hit_mask()
        return (
            hits_mask.sum(axis=1) / self.recommendations.shape[1]
        ).sum() / self.recommendations.shape[0]

    def _get_hit_mask(self) -> np.ndarray:
        return (
            self.holdout[..., None] ==
            np.expand_dims(self.recommendations, axis=1)
        ).any(axis=1)

In [38]:
metrics_carts = Metrics(K_RECS, np.array(recs_carts), test_carts)
metrics_carts.report()

HR@200 = 0.011
MRR@200 = 0.0006
Recall@200 = 0.0001


In [22]:
test_orders, recs_orders= prepare_data(df_orders_t, df_orders_train, K_TEST, K_RECS)
metrics_orders = Metrics(K_RECS, np.array(recs_orders), test_orders)
print(metrics_orders.hit_rate(), metrics_orders.mean_reciprocal_rank(), metrics_orders.recall())

NameError: name 'df_orders_train' is not defined

In [92]:
metrics_carts = Metrics(K_RECS, np.array(recs_orders), test_orders)
metrics_carts.report()

HR@50 = 0.7049
MRR@50 = 0.0665
Recall@50 = 0.5608


## TF-IDF Normalization

In [54]:
def prepare_data(df_test, df_recs, k_test, k_recs):
    valid_session = np.intersect1d(
        df_test["session"].unique(),
        df_recs["session"].unique()
    )

    df_test_l = set_ranks(df_test)
    df_test_l = df_test_l[df_test_l["session"].isin(valid_session)]
    df_test_l = df_test_l[df_test_l["rank"] <= k_test]
    df_test_l = df_test_l.groupby("session")["aid"].apply(list).reset_index().sort_values("session")

    df_test_l["aid"] = (
        df_test_l["aid"]
        .apply(lambda x: np.pad(
            x[:k_test],
            (0, max(k_test - len(x), 0)),
            mode="constant",
            constant_values=-1
        ))
    )
    test = np.stack(df_test_l["aid"].values, axis=0)
    print(test.shape)

    df_recs_l = df_recs[df_recs["session"].isin(valid_session)]
    df_recs_l["modified_pred"] = (
        df_recs_l["pred_tfidf"]
        .apply(lambda x: extract_n_elements(x, k_recs))
    )
    
    recs = df_recs_l["modified_pred"].tolist()

    return test, recs


In [55]:
K_TEST = 5
K_RECS = 200

In [56]:
test_carts, recs_carts = prepare_data(df_carts_r, df_carts_train, K_TEST, K_RECS)

(40093, 5)


C:\Users\tanni\AppData\Local\Temp\ipykernel_9620\2324496305.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recs_l["modified_pred"] = (


In [57]:
metrics_carts = Metrics(K_RECS, np.array(recs_carts), test_carts)
metrics_carts.report()

HR@200 = 0.7662
MRR@200 = 0.0307
Recall@200 = 0.5553


In [103]:
test_carts, recs_carts = prepare_data(df_orders_t, df_orders_train, K_TEST, K_RECS)
metrics_carts = Metrics(K_RECS, np.array(recs_carts), test_carts)
metrics_carts.report()

(10586, 5)
HR@200 = 0.8957
MRR@200 = 0.0687
Recall@200 = 0.7747


C:\Users\tanni\AppData\Local\Temp\ipykernel_5248\2324496305.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recs_l["modified_pred"] = (
